# Results analysis

In this notebook we will detail the experiments done on the chosen datasets (Basque and Japanesse, both taken from Universal Dependencies) and the obtained results.

In [7]:
from src.dataset_loader import Dataset
from pathlib import Path

basque_dataset = Dataset(
    dataset_name='UD_Basque-BDT',
    train_path=Path('../data/UD_Basque-BDT/eu_bdt-ud-train.conllu'),
    dev_path=Path('../data/UD_Basque-BDT/eu_bdt-ud-dev.conllu'),
    test_path=Path('../data/UD_Basque-BDT/eu_bdt-ud-test.conllu'),
)

japanese_dataset = Dataset(
    dataset_name='UD_Japanese-GSD',
    train_path=Path('../data/UD_Japanese-GSD/ja_gsd-ud-train.conllu'),
    dev_path=Path('../data/UD_Japanese-GSD/ja_gsd-ud-dev.conllu'),
    test_path=Path('../data/UD_Japanese-GSD/ja_gsd-ud-test.conllu'),
)

In [ ]:
from src.hmm import HiddenMarkovModel


def get_model_predictions(d):
    hmm = HiddenMarkovModel(d)
    gold = d.train.data[0]
    pred = hmm.predict([token for token, _ in d.train.data[0]])

    y_gold = [tag for word, tag in gold]
    y_pred = [tag for word, tag in pred[0]]

    print('Pred:', pred)
    print('Gold:', gold)

In [ ]:
get_model_predictions(basque_dataset.train.data[0])